In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
categorical_variables = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION',
                        'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[categorical_variables].values.reshape(-7,7)))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(categorical_variables)

In [6]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(categorical_variables, axis=1)

In [7]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y=application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=250)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Create a Random Forest Model to determine which variables are contributing to classification
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [10]:
# Fit the model and make predictions
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [11]:
# Check the accuracy score
accuracy = accuracy_score(y_test, predictions)
accuracy

0.7144023323615161

In [12]:
# Sort features by their importance
feature_importance = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
feature_importance

[(0.40380697426266887, 'ASK_AMT'),
 (0.13023419779942544, 'AFFILIATION_CompanySponsored'),
 (0.12597770599227456, 'AFFILIATION_Independent'),
 (0.030087436280445495, 'APPLICATION_TYPE_T5'),
 (0.029528566877538142, 'ORGANIZATION_Association'),
 (0.02423742811719354, 'APPLICATION_TYPE_T10'),
 (0.0212169061704939, 'CLASSIFICATION_C2100'),
 (0.018357875356204627, 'ORGANIZATION_Trust'),
 (0.018160287461598618, 'CLASSIFICATION_Other'),
 (0.017447504231304953, 'APPLICATION_TYPE_T4'),
 (0.017329904841551894, 'CLASSIFICATION_C2000'),
 (0.017259143692031298, 'APPLICATION_TYPE_T19'),
 (0.01396563704888165, 'CLASSIFICATION_C1000'),
 (0.013775944920041788, 'APPLICATION_TYPE_T3'),
 (0.012069888682018335, 'APPLICATION_TYPE_T6'),
 (0.010871228016337518, 'USE_CASE_ProductDev'),
 (0.010846150567662315, 'USE_CASE_Preservation'),
 (0.010068550827382948, 'CLASSIFICATION_C1200'),
 (0.009230471149833368, 'INCOME_AMT_0'),
 (0.007745611240037039, 'CLASSIFICATION_C3000'),
 (0.006790945597500714, 'INCOME_AMT_250

In [13]:
# Add features with little importance to a list
bad_features = []
for feature in feature_importance:
    if feature[0] < 0.001:
        bad_features.append(feature[1])
        
bad_features

['AFFILIATION_National',
 'ORGANIZATION_Corporation',
 'SPECIAL_CONSIDERATIONS_Y',
 'AFFILIATION_Regional',
 'SPECIAL_CONSIDERATIONS_N',
 'AFFILIATION_Other',
 'USE_CASE_Other',
 'STATUS']

In [14]:
X = X.drop(bad_features, axis=1)
X.head()

,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108590,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6692,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,142590,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=250)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_1 = tf.keras.models.Sequential()

# First hidden layer
nn_1.add(tf.keras.layers.Dense(units=10, activation='relu', input_dim=35))

# Second hidden layer
nn_1.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Third hidden layer
nn_1.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer
nn_1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                360       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 591
Trainable params: 591
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn_1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5956 - accuracy: 0.7061
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5610 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7293
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5547 - accuracy: 0.7299
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5529 - accuracy: 0.7309
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5525 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5516 - accuracy: 0.7320
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5495 - accura

804/804 [==============================] - 2s 3ms/step - loss: 0.5429 - accuracy: 0.7362
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 0.7343
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7363
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 0.7364
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5425 - accuracy: 0.7365
Epoch 86/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5427 - accuracy: 0.7367
Epoch 87/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5423 - accuracy: 0.7352
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5422 - accuracy: 0.7365
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5424 - accuracy: 0.7353
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5523 - accuracy: 0.7291 - 673ms/epoch - 3ms/step
Loss: 0.5523296594619751, Accuracy: 0.7290962338447571


In [20]:
# Attempt 2: Change the activation function and increase number of neurons per layer to 30

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation='tanh', input_dim=35))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation='tanh'))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(units=30, activation='tanh'))

# Output layer
nn_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 30)                1080      
                                                                 
 dense_5 (Dense)             (None, 30)                930       
                                                                 
 dense_6 (Dense)             (None, 30)                930       
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                                 
Total params: 2,971
Trainable params: 2,971
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5725 - accuracy: 0.7191
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7286
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5518 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7296
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7325
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7327
Epoch 10/100
804/804 [==============================] - 11s 14ms/step - loss: 0.5475 - accu

804/804 [==============================] - 2s 2ms/step - loss: 0.5353 - accuracy: 0.7393
Epoch 82/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7386
Epoch 83/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5352 - accuracy: 0.7393
Epoch 84/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7397
Epoch 85/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5352 - accuracy: 0.7398
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5350 - accuracy: 0.7395
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5348 - accuracy: 0.7397
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 0.7398
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5346 - accuracy: 0.7390
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy: 

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5564 - accuracy: 0.7296 - 662ms/epoch - 2ms/step
Loss: 0.5564249753952026, Accuracy: 0.7295626997947693


In [24]:
# Attempt 3: Go back to relu activation function and increase number of neurons per layer to 70

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(units=70, activation='relu', input_dim=35))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(units=70, activation='relu'))

# Third hidden layer
nn_3.add(tf.keras.layers.Dense(units=70, activation='relu'))

# Output layer
nn_3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 70)                2520      
                                                                 
 dense_9 (Dense)             (None, 70)                4970      
                                                                 
 dense_10 (Dense)            (None, 70)                4970      
                                                                 
 dense_11 (Dense)            (None, 1)                 71        
                                                                 
Total params: 12,531
Trainable params: 12,531
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Define the checkpoint path and filenames
checkpoint_path = "AlphabetSoupCharity_Optimization.h5"

In [26]:
# Compile the model
nn_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [28]:
# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=100,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5686 - accuracy: 0.7206
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5550 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5506 - accuracy: 0.7331
Epoch 5/100
795/804 [============================>.] - ETA: 0s - loss: 0.5490 - accuracy: 0.7331
Epoch 5: saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7332
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7333
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accuracy: 0.7341
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7348
Epoch 9/100
804/804 [=========================

Epoch 64/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5351 - accuracy: 0.7398
Epoch 65/100
784/804 [============================>.] - ETA: 0s - loss: 0.5357 - accuracy: 0.7394
Epoch 65: saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 2s 3ms/step - loss: 0.5351 - accuracy: 0.7399
Epoch 66/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5351 - accuracy: 0.7401
Epoch 67/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5353 - accuracy: 0.7401
Epoch 68/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5355 - accuracy: 0.7398
Epoch 69/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5351 - accuracy: 0.7397
Epoch 70/100
793/804 [============================>.] - ETA: 0s - loss: 0.5353 - accuracy: 0.7401
Epoch 70: saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 2s 3ms/step - loss: 0.5349 - accuracy:

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5771 - accuracy: 0.7299 - 503ms/epoch - 2ms/step
Loss: 0.5770583152770996, Accuracy: 0.729912519454956
